In [1]:
from __future__ import division, print_function
from keras import backend as K
from keras.applications import inception_v3
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, merge
from keras.layers.core import Activation, Dense, Dropout, Lambda
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from random import shuffle
from scipy.misc import imresize
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
%matplotlib inline

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


#### GPU

In [2]:
os.environ["CUDA_VISIBLE_DEVICE"] = "0" # setting wanna use GPU No.
os.system('echo $CUDA_VISIBLE_DEVICE')

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4 # setting GPU in-use memory
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
set_session(tf.Session(config=config))

#### To reset GPU memory

===========================================================================================================================

In [3]:
dfcols = ['img_name']
df = pd.DataFrame(columns=dfcols)

DATA_DIR = ""
IMAGE_DIR = os.path.join(DATA_DIR, "new_new_test_folder")
TUNE_DIR = os.path.join(DATA_DIR, "golden_finding", "F0T084627008A")

path = '/home/ubuntu/siamese/golden_finding/F0T084627008A' 
new_path = '/home/ubuntu/siamese/golden_finding/F0T084651003A_Pred/' 

BATCH_SIZE = 128

In [4]:
file_name = []

for img_file in os.listdir(TUNE_DIR):
    if img_file.find(".jpg") != -1:
        df = df.append(pd.Series([img_file], index=dfcols),
                            ignore_index=True) 
        
df = df.sort_values("img_name", ascending=True)
df = df.reset_index(drop=True)
# df.to_csv('all.csv', encoding='utf-8', index=False)

In [28]:
df

,img_name
0,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...
1,F0T084627008A_04682f0f-75cd-4de8-af56-a9f7afe0...
2,F0T084627008A_06380199-90b2-49e1-93fb-062b147e...
3,F0T084627008A_069819ed-da4e-4978-ab3f-ca0f6eec...
4,F0T084627008A_0992733f-2f4c-4775-b6c9-e9ed1f33...
5,F0T084627008A_0b561cb2-1371-4e08-b68c-119554c2...
6,F0T084627008A_0de9d39e-7c8c-48c8-b57e-b768642d...
7,F0T084627008A_0f341766-5aba-4243-a366-459065b1...
8,F0T084627008A_120d977c-e82f-465a-9c9b-e976f037...
9,F0T084627008A_12361acc-f3ad-42f8-baaa-0dd9bbc0...


In [6]:
mlabel = ''
all_triples = []
triples = []
for index_1, row_1 in df.iterrows():
    for index_2, row_2 in df.iterrows():
        if row_1['img_name'][13:] != row_2['img_name'][13:]:
            mlabel = '1'
            triples = []
            mlabel = '1'
            triples = [row_1['img_name'],row_2['img_name'],mlabel]
            all_triples.append(triples)

In [7]:
print(len(all_triples))

20880


In [8]:
df_pair = pd.DataFrame(all_triples)
# df_pair.to_csv('all_pair.csv', encoding='utf-8', index=False)

In [9]:
df_pair.head()

,0,1,2
0,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...,F0T084627008A_04682f0f-75cd-4de8-af56-a9f7afe0...,1
1,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...,F0T084627008A_06380199-90b2-49e1-93fb-062b147e...,1
2,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...,F0T084627008A_069819ed-da4e-4978-ab3f-ca0f6eec...,1
3,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...,F0T084627008A_0992733f-2f4c-4775-b6c9-e9ed1f33...,1
4,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...,F0T084627008A_0b561cb2-1371-4e08-b68c-119554c2...,1


In [10]:
def load_image_cache(image_cache, image_filename):
    image = plt.imread(os.path.join(TUNE_DIR, image_filename))
    image = imresize(image, (299, 299))
    image = image.astype("float32")
    image = inception_v3.preprocess_input(image)
    image_cache[image_filename] = image
    
image_cache = {}
num_pairs = len(all_triples)
for i, (image_filename_l, image_filename_r, _) in enumerate(all_triples):
    if i % 1000 == 0:
        print("images from {:d}/{:d} pairs loaded to cache".format(i, num_pairs))
    if image_filename_l not in image_cache:
        load_image_cache(image_cache, image_filename_l)
    if image_filename_r not in image_cache:
        load_image_cache(image_cache, image_filename_r)
print("images from {:d}/{:d} pairs loaded to cache, COMPLETE".format(i, num_pairs))

images from 0/20880 pairs loaded to cache


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


images from 1000/20880 pairs loaded to cache
images from 2000/20880 pairs loaded to cache
images from 3000/20880 pairs loaded to cache
images from 4000/20880 pairs loaded to cache
images from 5000/20880 pairs loaded to cache
images from 6000/20880 pairs loaded to cache
images from 7000/20880 pairs loaded to cache
images from 8000/20880 pairs loaded to cache
images from 9000/20880 pairs loaded to cache
images from 10000/20880 pairs loaded to cache
images from 11000/20880 pairs loaded to cache
images from 12000/20880 pairs loaded to cache
images from 13000/20880 pairs loaded to cache
images from 14000/20880 pairs loaded to cache
images from 15000/20880 pairs loaded to cache
images from 16000/20880 pairs loaded to cache
images from 17000/20880 pairs loaded to cache
images from 18000/20880 pairs loaded to cache
images from 19000/20880 pairs loaded to cache
images from 20000/20880 pairs loaded to cache
images from 20879/20880 pairs loaded to cache, COMPLETE


In [5]:
from keras.utils import multi_gpu_model

BEST_MODEL_FILE = os.path.join(DATA_DIR, "models", "A_A_inceptionv3-l1-f-best_100_args6.h5")
FINAL_MODEL_FILE = os.path.join(DATA_DIR, "models", "A_A_inceptionv3-l1-f-final_100_args6.h5")

print("==== Evaluation Results: best model on test set ====")
best_model = load_model(BEST_MODEL_FILE)
# parallel_model = multi_gpu_model(best_model, gpus=0)

==== Evaluation Results: best model on test set ====


BEST_MODEL_FILE = os.path.join(DATA_DIR, "models", "A_A_inceptionv3-l1-f-best_100_args6.h5")
FINAL_MODEL_FILE = os.path.join(DATA_DIR, "models", "A_A_inceptionv3-l1-f-final_100_args6.h5")

print("==== Evaluation Results: best model on test set ====")
best_model = load_model(BEST_MODEL_FILE)

In [12]:
def pair_generator(triples, image_cache, datagens, batch_size=32):
    while True:
        # shuffle once per batch
        indices = np.random.permutation(np.arange(len(triples)))
        num_batches = len(triples) // batch_size
        for bid in range(num_batches):
            batch_indices = indices[bid * batch_size : (bid + 1) * batch_size]
            batch = [triples[i] for i in batch_indices]
            X1 = np.zeros((batch_size, 299, 299, 3))
            X2 = np.zeros((batch_size, 299, 299, 3))
            Y = np.zeros((batch_size, 2))
            l_file , r_file, label_y = [], [], []
            for i, (image_filename_l, image_filename_r, label) in enumerate(batch):
                if datagens is None or len(datagens) == 0:
                    X1[i] = image_cache[image_filename_l]
                    X2[i] = image_cache[image_filename_r]
                else:
                    X1[i] = datagens[0].random_transform(image_cache[image_filename_l])
                    X2[i] = datagens[1].random_transform(image_cache[image_filename_r])
                Y[i] = [1, 0] if label == 0 else [0, 1]
                single_y = [1, 0] if int(label) == 0 else [0, 1]
                l_file.append(image_filename_l)
                r_file.append(image_filename_r)
                label_y.append(single_y)
            yield [X1, X2], label_y, [l_file,r_file]

In [13]:
from datetime import datetime
now_start = datetime.now()

In [14]:
i = 0

ytest, ytest_ , y_prob, l_file, r_file= [], [], [], [], []
test_pair_gen = pair_generator(all_triples, image_cache, None, BATCH_SIZE)
num_test_steps = len(all_triples) // BATCH_SIZE
print('total batch :', num_test_steps)
curr_test_steps = 0
for [X1test, X2test], Ytest, [file_l,file_r] in test_pair_gen:
    if curr_test_steps > num_test_steps:
        break
    Ytest_ = best_model.predict([X1test, X2test])
    ytest.extend(np.argmax(Ytest, axis=1).tolist())
    ytest_.extend(np.argmax(Ytest_, axis=1).tolist())
    y_prob.extend(Ytest_)
    l_file.extend(file_l)
    r_file.extend(file_r)

    curr_test_steps += 1
    i = i +1
    print(i)
acc = accuracy_score(ytest, ytest_)
cm = confusion_matrix(ytest, ytest_)

now_end = datetime.now()

print(now_start, now_end)
print("Accuracy Score: {:.3f}".format(acc))
print("Confusion Matrix")
print(cm)

total batch : 163
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
2018-08-10 03:10:08.092666 2018-08-10 03:49:40.959088
Accuracy Score: 0.986
Confusion Matrix
[[    0     0]
 [  291 20701]]


In [22]:
from pandas import Series, DataFrame

file_1 = DataFrame(l_file)
file_1.columns = ['l_filename']
file_r = DataFrame(r_file)
file_r.columns = ['r_filename']
mlabel = DataFrame(ytest)
mlabel.columns = ['Label']
pre_label = DataFrame(ytest_)
pre_label.columns = ['Pred_Label']
pro_label = DataFrame(y_prob)
pro_label.columns = ['net_pro','pos_pro']


golden_finding = pd.concat([file_1,file_r,mlabel,pre_label,pro_label] , axis = 1)
golden_finding.to_csv('Pred.csv', encoding='utf-8', index=False)

In [21]:
golden_finding['l_filename'] ='F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f4728f6.jpg'

In [19]:
golden_finding_pivot = pd.DataFrame()
golden_finding_pivot = golden_finding.groupby(by='l_filename').sum()
# golden_finding_pivot = golden_finding_pivot.drop(['Label', 'net_pro', 'pos_pro'], axis=1)
golden_finding_pivot = golden_finding_pivot.reset_index()
golden_finding_pivot['Pred_Label'] = golden_finding_pivot['Pred_Label'].apply(str)
golden_finding_pivot.to_csv('F0T084627008A.csv', encoding='utf-8', index=False)
golden_finding_pivot.head()

,l_filename,Label,Pred_Label,net_pro,pos_pro
0,F0T084627008A_0259cb3f-0330-4e39-9ba9-bfe71f47...,144,143,1.303664,142.696336
1,F0T084627008A_04682f0f-75cd-4de8-af56-a9f7afe0...,145,144,1.355626,143.644375
2,F0T084627008A_06380199-90b2-49e1-93fb-062b147e...,144,143,1.291287,142.708713
3,F0T084627008A_069819ed-da4e-4978-ab3f-ca0f6eec...,144,143,1.231908,142.768092
4,F0T084627008A_0992733f-2f4c-4775-b6c9-e9ed1f33...,146,145,1.300013,144.699986


In [85]:
# golden_finding_pivot = golden_finding_pivot.sort_values(by=['Pred_Label', 'pos_pro'], ascending=False)
print(golden_finding_pivot.iloc[:1, :1])

                 l_filename
28  _F0T084651003A_0029.jpg


In [71]:
from shutil import copyfile
new_filename = []
all_new_filename = []

for index, row in golden_finding_pivot.iterrows():
    sourcename = path + '/' + row['l_filename']
    
    new_filename = row['Pred_Label'] + row['l_filename']
    targetname = new_path + new_filename

#     print(new_filename)
    all_new_filename.append(new_filename)
    copyfile(sourcename,targetname)
    

golden_finding_pivot['new_filename'] = all_new_filename
print(len(all_new_filename))
golden_finding.to_csv('F0T084651003A_Re.csv', encoding='utf-8', index=False)

314


=============================================================================

In [ ]:
from datetime import datetime
now = datetime.now()

In [7]:
now1 = datetime.now()

image_1 = plt.imread(os.path.join(IMAGE_DIR ,'010001.jpg'))
# plt.imshow(image_1)
image_1 = imresize(image_1, (299, 299))
image_1 = image_1.astype("float32")
image_1 = inception_v3.preprocess_input(image_1)
# plt.imshow(image_1)


image_2 = plt.imread(os.path.join(IMAGE_DIR ,'110001.jpg'))
# plt.imshow(image_2)
image_2 = imresize(image_2, (299, 299))
image_2 = image_2.astype("float32")
image_2 = inception_v3.preprocess_input(image_2)
# plt.imshow(image_2)


image_1 = image_1.reshape(1,299,299,3)
image_2 = image_2.reshape(1,299,299,3)


Ytest_ = best_model.predict([image_1, image_2])
# pass
Ytest_

now2 = datetime.now()
print(now1, now2)

/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]


2018-07-25 04:00:54.657092 2018-07-25 04:00:56.841560


In [8]:
Ytest_

array([[ 0.97724462,  0.02275536]], dtype=float32)